In [ ]:
# File to DB 
# CSV to table  
# CSV 파일은 콤마를 기준으로 열단위 자료를 작성하는 파일로 데이터베이스의 테이블 구조와 호환성이 높다
 
# 1단계: 테이블 생성
import pandas as pd 
import pymysql
import os 

print(os.getcwd()) #파일경로확인 
bmi= pd.read_csv("data/bmi.csv") #csv파일 업로드 
bmi.info() 
print(type(bmi)) # 데이터프레임 형태 <class 'pandas.core.frame.DataFrame'>

height= bmi['height']
weight= bmi['weight']
label= bmi['label']

config = {  
    'host':'127.0.0.1',
    'user': 'scott', 
    'password': '1234', 
    'database': 'work',
    'port': 3306, 
    'charset':'utf8',      
    'use_unicode': True  # utf는 가변길이, 유니코드는 2byte 이상의 multi-byte # 해당 부분을 안쓰면 ascii code를 사용해야 함  
          }

try: 
    conn= pymysql.connect(**config)
    cursor = conn.cursor()  

    # 테이블 확인(show tables)
    cursor.execute("show tables")
    tables= cursor.fetchall()
    print(tables, type(tables)) # (('goods',),), 튜플타입으로 변환 
    
    # 스위칭 기법 
    sw = False  
    for table in tables: 
        if table[0]=='bmi_tab': 
            sw= True  # 테이블이 존재하는 경우 sw가 True 로 전환 
    
    # 테이블 실행 
    if not sw:
        print("테이블 없음")
        sql= ''' 
        create table bmi_tab(   
            height int not null,
            weight int not null, 
            label varchar(15) not null 
        )
        ''' 
        cursor.execute(sql)
    
    # 레코드 조회 
    cursor.execute("select * from bmi_tab")
    rows = cursor.fetchall()  
    
    if rows: #레코드가 있으면 실행, 
        for row in rows:
             print(f"{row[0]}\t{row[1]}\t{row[2]}")
        print("전체 레코드수:",len(rows))
        
    else: 
        print("100개 레코드 추가") 
        for i in range(100): 
            h= height[i]
            w= weight[i]
            l= label[i]
            cursor.execute(f"insert into bmi_tab values({h},{w},'{l}')   ")
            conn.commit() 
    
    
except Exception as e: 
    print("DB연동에러:",e)

finally: 
    conn.close() 
    cursor.close() 
    

In [ ]:
# score.csv 파일을 DB에 저장하세요
# table 명 score_tab으로 생성하세요 

import pandas as pd 
import pymysql

score= pd.read_csv("data/score.csv") 
#print(score.info)
#print(len(score))
#print(no[0],kor[0],eng[0])

no= score['no']
kor= score['kor']
eng = score['eng']
mat= score['mat'] 
dept= score['dept']

config = {  
    'host':'127.0.0.1',
    'user': 'scott', 
    'password': '1234', 
    'database': 'work',
    'port': 3306, 
    'charset':'utf8',      
    'use_unicode': True  # utf는 가변길이, 유니코드는 2byte 이상의 multi-byte # 해당 부분을 안쓰면 ascii code를 사용해야 함  
          }

try: 
    #DB연결객체 및 커서객체 생성 
    conn=pymysql.connect(**config)
    cursor= conn.cursor()  
    
    #테이블생성  
    sql = ''' 
        create table if not exists score_tab (  
        no int primary key, 
        kor int not null, 
        eng int not null, 
        mat int not null, 
        dept int not null  
        )
        ''' 
    cursor.execute(sql)
    cursor.execute("select * from score_tab")
    rows = cursor.fetchall()
    
    if rows: 
        print("번호\t국어\t영어\t수학\t반")
        for row in rows: 
            print(f"{row[0]}\t{row[1]}\t{row[2]}\t{row[3]}\t{row[4]}")
        print("전체 레코드수:", len(rows))
            
    else: # 레코드 없는경우 
        print("score.csv 파일을 DB에 추가")
        
        for i in range(len(score)):  
            sql =f"insert into score_tab values({no[i]},{kor[i]},{eng[i]},{mat[i]},{dept[i]})"
            cursor.execute(sql)
            conn.commit() 
            
except Exception as e: 
    print("DB연동에러:",e) 
    conn.rollback() 
     
finally: 
    conn.close() 
    cursor.close()  


In [ ]:
## json to table
# json 은 {키:값} 형식으로 제공하는 파일. 
# 1. json 형식으로 데이터를 디코딩
# 2. 데이터프레임 형식으로 변경하면 테이블 구조의 컬럼단위 저장 

In [ ]:
import pandas as pd
import pymysql
import json

# test= pd.read_json("data/labels.json") 
file = open("data/labels.json","r", encoding="utf8") 

# 디코딩: json -> object(python) 
lines= json.load(file)
print(type(lines)) #<class 'list'>
print(len(lines)) #요소 30개
print(lines[0]) #{'id': 76811, 'url': 'https://api.github.com/repos/pandas-dev/pandas/labels/Bug', 'name': 'Bug', 'color': 'e10c02', 'default': False}
print(type(lines[0])) # <class 'dict'>

# DataFrame 생성 
df= pd.DataFrame(lines)
print(df.info())
print(df.head())

# DB작업을 진행 

config = {  
    'host':'127.0.0.1',
    'user': 'scott', 
    'password': '1234', 
    'database': 'work',
    'port': 3306, 
    'charset':'utf8',      
    'use_unicode': True  # utf는 가변길이, 유니코드는 2byte 이상의 multi-byte # 해당 부분을 안쓰면 ascii code를 사용해야 함  
          }

try: 
    #DB연결객체 및 커서객체 생성 
    conn=pymysql.connect(**config)
    cursor= conn.cursor()  
    
    sql= '''create table if not exists labels ( 
        uid int not null,
        url varchar(150) not null,
        name varchar(50) not null,  
        color varchar(50) not null, 
        de char(5) 
    )'''
    cursor.execute(sql)
    
    #레코드 조회 후 있는경우 출력 없으면 | 추가  
    cursor.execute("select * from labels")
    rows= cursor.fetchall() 
    
    if rows: # 출력 
        print("labels 레코드 조회")
        for row in rows: 
            print(row) 
        print("전체 레코드 수:",len(rows))
    else:    # 추가 
        for i in range(len(df)): 
            uid= df.id[i] #df.columns
            url= df.url[i]
            name= df.name[i] 
            color = df.color[i]
            de= df.default[i]
            sql= f'''insert into labels values({uid},'{url}','{name}','{color}','{de}')
            ''' 
            cursor.execute(sql)
            conn.commit() 
            
            
    

except Exception as e:
    print("DB연동에러:",e) 
    conn.rollback()
    
finally: 
    cursor.close()
    conn.close() 



In [ ]:
print(df.columns) 
print(df.index)
print(df.values) 

In [96]:
# emp.csv 파일을 처리하여 JSON 데이터로 변환 
# emp.csv -> dict -> JSON 
# 저장된 JSON파일을 불러와서 DB 작업을 하세요 
import pandas as pd 
import json 

emp= pd.read_csv("data/emp.csv")

lst = [ {'No':int(emp.No[i]), 'Name': emp.Name[i],'Pay': int(emp.Pay[i])} for i in range(len(emp)) ]
lst #리스트가 감싸는 딕셔너리의 형태로 전환 

#dump -> json 형태의 문자열로 만들어줌 

type(emp.No[0]) # numpy.int64 
print(json.dumps(lst,ensure_ascii=False,indent=4))

with open('data/emp.json',"w",encoding="utf-8") as f: 
    json.dump(lst,f,ensure_ascii=False, indent=4 ) 

emp_j= pd.read_json("data/emp.json")
emp_j

df= pd.DataFrame(emp_j)

config = {  
    'host':'127.0.0.1',
    'user': 'scott', 
    'password': '1234', 
    'database': 'work',
    'port': 3306, 
    'charset':'utf8',      
    'use_unicode': True  # utf는 가변길이, 유니코드는 2byte 이상의 multi-byte # 해당 부분을 안쓰면 ascii code를 사용해야 함  
          }

try: 
    #DB연결객체 및 커서객체 생성 
    conn=pymysql.connect(**config)
    cursor= conn.cursor()  
    
    sql= '''create table if not exists emp_tab ( 
        no int not null,
        name varchar(50) not null, 
        pay int not null 
    )'''
    cursor.execute(sql)
    
    #레코드 조회 후 있는경우 출력 없으면 | 추가  
    cursor.execute("select * from emp_tab")
    rows= cursor.fetchall() 
    
    if rows: # 출력 
        print("emp 레코드 조회")
        for row in rows: 
            print(row) 
        print("전체 레코드 수:",len(rows))
    else:    # 추가 
        for i in range(len(df)): 
            no= df.No[i] #df.columns
            name= df.Name[i]
            pay= df.Pay[i] 
           
            sql= f'''insert into emp_tab values({no},'{name}','{pay}')''' 
            cursor.execute(sql)
            conn.commit() 
            
            
    

except Exception as e:
    print("DB연동에러:",e) 
    conn.rollback()
    
finally: 
    cursor.close()
    conn.close() 


[
    {
        "No": 101,
        "Name": "홍길동",
        "Pay": 150
    },
    {
        "No": 102,
        "Name": "이순신",
        "Pay": 450
    },
    {
        "No": 103,
        "Name": "강감찬",
        "Pay": 500
    },
    {
        "No": 104,
        "Name": "유관순",
        "Pay": 350
    },
    {
        "No": 105,
        "Name": "김유신",
        "Pay": 400
    }
]
